In [98]:
import pandas as pd
import seaborn as sns
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [99]:
df_values = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')
df_labels = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')


drop_columns = ['id', 'funder', 'num_private', 'longitude', 'latitude', 'wpt_name', 'subvillage', 'region_code', 'lga',
                'ward','recorded_by', 'scheme_name', 'extraction_type_group', 'payment', 'quality_group', 
                'quantity_group', 'source_type', 'source_class', 'waterpoint_type_group']

continuous_columns = ['amount_tsh', 'date_recorded', 'gps_height', 'population', 'construction_year']

categorical_columns = ['installer', 'basin', 'region', 'district_code', 'public_meeting',
                      'scheme_management', 'permit', 'extraction_type', 'extraction_type_class', 'management', 
                      'management_group', 'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type']

df_original = pd.merge(df_values, df_labels, on = 'id', how = 'inner')

df_original.drop(drop_columns, axis = 1, inplace = True)

df_original.columns

Index(['amount_tsh', 'date_recorded', 'gps_height', 'installer', 'basin',
       'region', 'district_code', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_class', 'management', 'management_group',
       'payment_type', 'water_quality', 'quantity', 'source',
       'waterpoint_type', 'status_group'],
      dtype='object')

### Clean data/one hot encoding
this portion will look at all the object data points and see whether or not they should be one hot encoded and/or binned.
1. filter out all the non-object data types and count the number of unique values to see if they would be uselful for classification. 
2. Delete columns that have significant overlap or are not good for one-hot-encoding
3. For date of recording, we plan on turning this into an interger by subtracting every date from the most recent date to get a value for how many dates have passed.

In [100]:
uvdict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    values_list = df_original[column].unique()
    uvdict[column] = len(values_list)

uvdict

{'date_recorded': 356,
 'installer': 2146,
 'basin': 9,
 'region': 21,
 'public_meeting': 3,
 'scheme_management': 13,
 'permit': 3,
 'extraction_type': 18,
 'extraction_type_class': 7,
 'management': 12,
 'management_group': 5,
 'payment_type': 7,
 'water_quality': 8,
 'quantity': 5,
 'source': 10,
 'waterpoint_type': 7,
 'status_group': 3}

In [101]:
# returns the number of nonvalues in the dictionary
nan_dict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    nan_dict[column] = df_original[column].isna().sum()

nan_dict

{'date_recorded': 0,
 'installer': 3655,
 'basin': 0,
 'region': 0,
 'public_meeting': 3334,
 'scheme_management': 3877,
 'permit': 3056,
 'extraction_type': 0,
 'extraction_type_class': 0,
 'management': 0,
 'management_group': 0,
 'payment_type': 0,
 'water_quality': 0,
 'quantity': 0,
 'source': 0,
 'waterpoint_type': 0,
 'status_group': 0}

'recorded_by', 'funder', 'scheme_name' were the three columns to delete outright. Funder was removed because it was nearly identical to installer but contained fewer unique values. For scheme_name, nearly half the data was missing and it seem to relate to scheme management, and 'recorded_by' only has one value making it useless for predictions.

In [102]:
nan_dict = {}

for column in df_original.select_dtypes(exclude=['int','float']):
    nan_dict[column] = df_original[column].isna().sum()

nan_dict

{'date_recorded': 0,
 'installer': 3655,
 'basin': 0,
 'region': 0,
 'public_meeting': 3334,
 'scheme_management': 3877,
 'permit': 3056,
 'extraction_type': 0,
 'extraction_type_class': 0,
 'management': 0,
 'management_group': 0,
 'payment_type': 0,
 'water_quality': 0,
 'quantity': 0,
 'source': 0,
 'waterpoint_type': 0,
 'status_group': 0}

In [103]:
# replace NaN values with 'not_listed string'
df_original.loc[:, ['installer', 'scheme_management','public_meeting','permit']] = df_original.loc[:, ['installer', 'scheme_management','subvillage','public_meeting','permit']].fillna('Not known')


/Users/lucasblasdel/opt/anaconda3/envs/learn-env/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [104]:
# replace values with fewer than 200
threshold = 200
cols = ['installer']

for col in cols:
    valued_dict = dict(df_original[col].value_counts())
    safe_values = list(key for key, value in valued_dict.items() if value >= threshold)
#     replace_values = list(filter(lambda x: x not in safe_values, all_values))
    df_original.loc[:, col] = df_original.loc[:, col].map(lambda y: 'other' if y not in safe_values else y)

df_original.installer.value_counts()

other                         19710
DWE                           17402
Not known                      3672
Government                     1825
RWE                            1206
Commu                          1060
DANIDA                         1050
KKKT                            898
Hesawa                          840
0                               777
TCRS                            707
Central government              622
CES                             610
Community                       553
DANID                           552
District Council                551
HESAWA                          539
World vision                    408
LGA                             408
WEDECO                          397
TASAF                           396
District council                392
Gover                           383
AMREF                           329
TWESA                           316
WU                              301
Dmdd                            287
ACRA                        

In [105]:
basedate = datetime(2020, 1, 1)
a = list(range(1955,2016,5))
cut_bins = [-1]
cut_bins.extend(a)
cut_labels = ['not available', '56-60','61-65','66-70','71-75','76-80','81-85','86-90','91-95','96-00','01-05','06-10','11-15']
df_original.loc[:, 'construction_year_bin'] = pd.cut(df_original['construction_year'], bins = cut_bins, labels = cut_labels)

df_original.construction_year_bin

0                96-00
1                06-10
2                06-10
3                86-90
4        not available
             ...      
59395            96-00
59396            96-00
59397    not available
59398    not available
59399            01-05
Name: construction_year_bin, Length: 59400, dtype: category
Categories (13, object): [not available < 56-60 < 61-65 < 66-70 ... 96-00 < 01-05 < 06-10 < 11-15]

In [106]:
df_original['days_since_recording'] = df_original.loc[:,'date_recorded'].map(lambda x: (basedate - datetime.strptime(x, "%Y-%m-%d")).days)



In [107]:
df_original.district_code = df_original.district_code.astype('str')

In [108]:
new_drop_list = ['date_recorded', 'construction_year']

df_cleaned = df_original.drop(new_drop_list, axis = 1)
df_cleaned

,amount_tsh,gps_height,installer,basin,region,district_code,population,public_meeting,scheme_management,permit,...,management,management_group,payment_type,water_quality,quantity,source,waterpoint_type,status_group,construction_year_bin,days_since_recording
0,6000.0,1390,other,Lake Nyasa,Iringa,5,109,True,VWC,False,...,vwc,user-group,annually,soft,enough,spring,communal standpipe,functional,96-00,3215
1,0.0,1399,other,Lake Victoria,Mara,2,280,Not known,Other,True,...,wug,user-group,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,06-10,2492
2,25.0,686,World vision,Pangani,Manyara,4,250,True,VWC,True,...,vwc,user-group,per bucket,soft,enough,dam,communal standpipe multiple,functional,06-10,2501
3,0.0,263,UNICEF,Ruvuma / Southern Coast,Mtwara,63,58,True,VWC,True,...,vwc,user-group,never pay,soft,dry,machine dbh,communal standpipe multiple,non functional,86-90,2529
4,0.0,0,other,Lake Victoria,Kagera,1,0,True,Not known,True,...,other,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional,not available,3094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,CES,Pangani,Kilimanjaro,5,125,True,Water Board,True,...,water board,user-group,per bucket,soft,enough,spring,communal standpipe,functional,96-00,2434
59396,4700.0,1212,other,Rufiji,Iringa,4,56,True,VWC,True,...,vwc,user-group,annually,soft,enough,river,communal standpipe,functional,96-00,3161
59397,0.0,0,Not known,Rufiji,Mbeya,7,0,True,VWC,False,...,vwc,user-group,monthly,fluoride,enough,machine dbh,hand pump,functional,not available,3187
59398,0.0,0,other,Rufiji,Dodoma,4,0,True,VWC,True,...,vwc,user-group,never pay,soft,insufficient,shallow well,hand pump,functional,not available,3221


### One hot encoding time

In [109]:

df_objects = df_cleaned.select_dtypes(exclude=['int','float']).drop(['status_group'], axis = 1)
df_nums = df_cleaned.select_dtypes(include=['int','float'])

df_onehot = pd.get_dummies(df_objects)

df_final = pd.concat([df_nums, df_onehot], axis = 1)


In [110]:
df_temp = df_objects.drop(['construction_year_bin'], axis = 1)

enc = OneHotEncoder()
enc.fit(df_temp)


TypeError: argument must be a string or number

In [111]:
df_final

X = df_final
y = df_cleaned['status_group']
# y = list(map(lambda x: 1 if x == 'functional' else 0, df_cleaned['status_group']))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .9, random_state = 42)

scaler = MinMaxScaler()
X_train_ = scaler.fit_transform(X_train)
X_test_ = scaler.transform(X_test)

y

0            functional
1            functional
2            functional
3        non functional
4            functional
              ...      
59395        functional
59396        functional
59397        functional
59398        functional
59399        functional
Name: status_group, Length: 59400, dtype: object

In [112]:
LR = LogisticRegression(random_state = 0)

In [113]:
LR.fit(X_train_, y_train)

/Users/lucasblasdel/opt/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lucasblasdel/opt/anaconda3/envs/learn-env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [114]:
preds = LR.predict(X_test_)

accuracy_score(y_test, preds)

0.7557239057239057